In [64]:
from finance.mongo.extraction import query_mongodb
from finance.params import *
import numpy as np
import pandas as pd


In [61]:
exchange_ls = ['PNK', 'NASDAQ']
table_name = 'company_profile'


In [75]:
result = query_mongodb(exchange_ls=exchange_ls, table=table_name)

exchange {} - 22906


In [63]:
result[0]['description'] == None

True

In [76]:
result_df = pd.DataFrame(result)
result_df

,_id,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,657dc9f9b2cffe5b07e21e8e,SSSSL,23.3966,0.000000,1214,1.070344e+08,1.50,22.28-25.0,0.1516,SuRo Capital Corp. 6.00% Notes due 2026,...,None,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-12-17,True,False,True,False,False
1,657dc9f9b2cffe5b07e21e8f,LVAC,10.4300,0.000000,44325,1.499312e+08,0.00,9.94-10.85,0.0000,LAVA Medtech Acquisition Corp.,...,02451,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-12-17,False,False,False,False,False
2,657dc9fab2cffe5b07e21e90,CPLP,13.8000,0.904000,38893,2.753293e+08,0.60,11.93-16.1,0.0500,Capital Product Partners L.P.,...,18537,-124.48770,138.809204,https://financialmodelingprep.com/image-stock/...,2007-03-30,True,False,True,False,False
3,657dc9fdb2cffe5b07e21e96,CMTL,7.9200,1.474000,276725,2.255537e+08,0.40,7.26-16.87,0.0600,Comtech Telecommunications Corp.,...,11747,2.18848,12.671500,https://financialmodelingprep.com/image-stock/...,1980-03-17,False,False,True,False,False
4,657dc9feb2cffe5b07e21e98,VLON,0.4000,-1.148559,8091680,5.392920e+06,0.00,0.2054-16.5,-0.0600,"Vallon Pharmaceuticals, Inc.",...,19103,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-02-10,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22901,6583182f570a04d552681823,FLLLF,0.0008,2.968000,2351,4.243100e+04,0.00,8.0E-4-0.0566,-0.0018,Ultra Brands Ltd.,...,V6C 0A6,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2018-07-06,True,False,True,False,False
22902,65831830570a04d552681824,FAURY,5.0000,1.986000,0,4.451885e+09,0.00,5.0-5.0,0.0000,Faurecia S.E.,...,92000,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,,True,False,False,False,False
22903,65831830570a04d552681825,TKPHF,27.4800,0.643000,30753,4.311351e+10,1.39,26.25-34.47,-0.5600,Takeda Pharmaceutical Company Limited,...,103-8668,-153.13138,180.611376,https://financialmodelingprep.com/image-stock/...,2008-06-25,True,False,True,False,False
22904,65831830570a04d552681826,XANAF,0.0420,0.366000,1634,7.143444e+07,0.00,0.0033-0.11,-0.0280,Xanadu Mines Limited,...,2000,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2014-02-24,True,False,True,False,False


In [89]:
result_df.dropna(subset='description', inplace=True)
result_df.dropna(subset=['sector', 'industry'], inplace=True)
result_df.drop_duplicates(subset='symbol', inplace=True)

In [90]:
result_df.shape

(13513, 37)

In [91]:
result_description = result_df.to_dict('records')

In [92]:
import chromadb
#chroma_client = chromadb.PersistentClient(path="../chroma/")
chroma_client = chromadb.Client()

In [93]:
collection = chroma_client.create_collection(name="general_profile")

UniqueConstraintError: Collection general_profile already exists

In [94]:
documents = [i['description'] for i in result_description]
ids = [i['symbol'] for i in result_description]
metadatas = [{'symbol':i['symbol'], 'exchangeShortName':i['exchangeShortName'], \
            'industry':i['industry'], 'sector':i['sector']}\
        for i in result_description]

In [98]:
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids)

In [99]:
chroma_client.get_collection(name="general_profile").peek() 

{'ids': ['AABVF',
  'AACG',
  'AACI',
  'AACIU',
  'AACQU',
  'AACS',
  'AADI',
  'AAEEF',
  'AAFRF',
  'AAGC'],
 'embeddings': [[0.022096838802099228,
   -0.04768630862236023,
   0.008979343809187412,
   0.0018998837331309915,
   0.02437485195696354,
   -0.07804165780544281,
   0.023293135687708855,
   -0.02596047706902027,
   0.025118649005889893,
   -0.012150503695011139,
   -0.01831497624516487,
   -0.005963717587292194,
   -0.07856880128383636,
   -0.04076714068651199,
   -0.006789329927414656,
   -0.03230869024991989,
   -0.005617864895612001,
   -0.03250986710190773,
   -0.04849237948656082,
   0.009162010625004768,
   0.032378215342760086,
   -0.09041781723499298,
   -0.004255339037626982,
   -0.06960277259349823,
   0.07791176438331604,
   0.12416034936904907,
   0.054858624935150146,
   0.015946201980113983,
   0.013804895803332329,
   0.05157579109072685,
   0.001774816308170557,
   -0.07786058634519577,
   -0.0015835168305784464,
   -0.007382535375654697,
   -0.007533122319

In [105]:
results = collection.query(
    query_texts=["says a late-stage trial of its candidate drug for an autoimmune condition f"],
    n_results=10
)
results

{'ids': [['ARGX',
   'IMVT',
   'APLS',
   'ANNX',
   'OMER',
   'CABA',
   'IMMP',
   'AVTX',
   'ALT',
   'SLRN']],
 'distances': [[0.9233061075210571,
   0.938607931137085,
   0.9473900198936462,
   0.9495989680290222,
   0.9710171222686768,
   1.0093952417373657,
   1.0243687629699707,
   1.0271024703979492,
   1.0272918939590454,
   1.0275328159332275]],
 'metadatas': [[{'exchangeShortName': 'NASDAQ',
    'industry': 'Biotechnology',
    'sector': 'Healthcare',
    'symbol': 'ARGX'},
   {'exchangeShortName': 'NASDAQ',
    'industry': 'Biotechnology',
    'sector': 'Healthcare',
    'symbol': 'IMVT'},
   {'exchangeShortName': 'NASDAQ',
    'industry': 'Biotechnology',
    'sector': 'Healthcare',
    'symbol': 'APLS'},
   {'exchangeShortName': 'NASDAQ',
    'industry': 'Biotechnology',
    'sector': 'Healthcare',
    'symbol': 'ANNX'},
   {'exchangeShortName': 'NASDAQ',
    'industry': 'Biotechnology',
    'sector': 'Healthcare',
    'symbol': 'OMER'},
   {'exchangeShortName': 'NASD